# Visual and Interactive Data Exploration

In [121]:
# authentication: put in alberto's file
  # standardized authentication
# publish maps from cf account  
from cartoframes.auth import set_default_credentials

set_default_credentials('cartoframes', api_key='de10d837671b45c192897b3c239e4d1d1a958f6c')

In [125]:
# Add layers
from cartoframes.viz import Map, Layer

Map([
    Layer('starbucks_brooklyn_iso_enriched'),
    Layer('starbucks_brooklyn_geocoded')
])

In [108]:
import pandas

df = pandas.read_csv('../files/starbucks_brooklyn.csv')
df




FileNotFoundError: [Errno 2] File b'../files/starbucks_brooklyn.csv' does not exist: b'../files/starbucks_brooklyn.csv'

Add widgets and popups to isochrone Layer
- formula widgets on demographic variables
- add hover on store_id

Use size continuous layer to symbolize store points by revenue
- by default, you see that the points are now sized proportionate to their revenue values 
- a continuous point legend has drawn on the map
- and if you hover over each point, you will see its revenue value

This helps us to visually and interactively explore the relationship between revenue and our demographic variables by each location. You will notice as you zoom and pan the map, the widget values are updated to summarize the current view.

Let's make a few adjustments to more easily distinguish between the highest and lowest performing store
- first, let's customize the default symbol size range since the smallest value is hard to see (`[10,50]`)
- next, let's give the legend a more appropriate title which will also be the title for the popup (title='Annual Revenue'($))
- and finally, let's add a histogram widget also based on revenue by setting `widget=True`

Insights:
- using this map, you can explore your data visually and interactively in different ways. 

For example, you can now more easily locate higher and lower performing stores based on their symbol size, zoom in to summarize demographic characteristics, and hover to see the store_id. Alternatively, you could select a range of revenues from the histogram widget to see which stores meet the criteria. You can also use the Store ID category widget to isolate and summarize values for a particular store.

- Try and see if you can find the highest and lowest performing stores and summarize the demographic characteristics of each one!


In [126]:
# Add widgets and popups to explore data
# Size points by revenue
    
from cartoframes.viz.widgets import formula_widget, histogram_widget, category_widget
from cartoframes.viz import Popup
from cartoframes.viz.helpers import size_continuous_layer

Map([
    Layer(
        'starbucks_brooklyn_iso_enriched',
        widgets=[
            formula_widget(
                'popcy',
                'sum',
                title='Total Population Served'
            ),
            formula_widget(
                'inccymedhh',
                'sum',
                title='Median Income ($)'
            ),
            formula_widget(
                'lbfcyempl',
                'sum',
                title='Employed Population',
            ),
            formula_widget(
                'educybach',
                'sum',
                title='Number of People with Bachelor Degree',
            ),
            category_widget(
                'store_id',
                title='Store ID'
            )
        ],
        popup=Popup({
            'hover': {
                'title': 'Store ID',
                'value': '$store_id'
            }
        })
      ),
    size_continuous_layer(
        'starbucks_brooklyn_geocoded',
        'revenue',
        size=[10,50],
        title='Annual Revenue ($)',
        widget=True
    )
])

Now that you have gained some insight into the relationship between revenue and demographics, let's say that the most influential factor of how well a store performed was median income and you want your final map to symbolize that.

Use the color_bins layer to color the isochrones by that value
Make some adjustments to the style of the points to account for the colored polygons underneath

Add the store points for reference with a hover to identify a store's id and address

In [129]:
from cartoframes.viz.helpers import color_bins_layer

Map([
    color_bins_layer(
        'starbucks_brooklyn_iso_enriched',
        'inccymedhh',
        bins=7,
        palette='pinkyl',
        opacity=0.8,
        stroke_width=0,
        title='Median Household Income ($)',
        footer='Source: US Census Bureau'
    ),
    size_continuous_layer(
        'starbucks_brooklyn_geocoded',
        'revenue',
        size=[10,50],
        range_max=1000000,
        opacity=0,
        stroke_color='turquoise',
        stroke_width=2,
        title='Annual Revenue ($)',
        description='Reported in 2018'
    ),
    Layer(
        'starbucks_brooklyn_geocoded',
        popup=Popup({
            'hover': {
                'title': 'Store ID',
                'value': '$store_id'
            }
        })
        
    ),
])

Set viewport to focus on the stores with similar incomes but the highest and lowest revenues

In [159]:
from cartoframes.viz import basemaps

Map([
    color_bins_layer(
        'starbucks_brooklyn_iso_enriched',
        'inccymedhh',
        bins=7,
        palette='pinkyl',
        stroke_width=0,
        title='Median Household Income ($)',
        footer='Source: US Census Bureau'
    ),
    size_continuous_layer(
        'starbucks_brooklyn_geocoded',
        'revenue',
        size=[10,50],
        range_max=1000000,
        opacity=0,
        stroke_color='turquoise',
        stroke_width=2,
        title='Annual Revenue ($)',
        description='Reported in 2018'
    ),
    Layer(
        'starbucks_brooklyn_geocoded',
        popup=Popup({
            'hover': {
                'title': 'Store ID',
                'value': '$store_id'
            }
        })
        
    ),
], 
    show_info=True,viewport={'zoom': 12.00, 'lat': 40.649062, 'lng': -73.966577}
)

In [163]:
from cartoframes.viz import Layout

Layout([
    Map([
        size_continuous_layer(
            'starbucks_brooklyn_geocoded',
            'revenue',
            size=[10,50],
            range_max=1000000,
            opacity=0,
            stroke_color='turquoise',
            stroke_width=2,
            title='Annual Revenue'
        ),
        Layer('starbucks_brooklyn_geocoded')
    ]),
    Map([
        color_bins_layer(
            'starbucks_brooklyn_iso_enriched',
            'inccymedhh',
            bins=7,
            palette='pinkyl',
            stroke_width=0,
            title='Median Income',
        ),
        Layer('starbucks_brooklyn_geocoded'),
    ]),
    Map([
        color_bins_layer(
            'starbucks_brooklyn_iso_enriched',
            'popcy',
            bins=7,
            palette='pinkyl',
            stroke_width=0,
            title='Total Pop',
        ),
        Layer('starbucks_brooklyn_geocoded'),
    ]),
    Map([
        color_bins_layer(
            'starbucks_brooklyn_iso_enriched',
            'lbfcyempl',
            bins=7,
            palette='pinkyl',
            stroke_width=0,
            title='Employed Pop',
        ),
        Layer('starbucks_brooklyn_geocoded'),
    ]),
],2,2,viewport={'zoom': 10, 'lat': 40.64, 'lng': -73.92}, map_height=400)
    
